<a href="https://colab.research.google.com/github/AzucenaMV/CapstoneProject/blob/master/DataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pickle
import pandas as pd
import numpy as np

In [0]:
data_dict = pickle.load( open( "data_dict.pickle", "rb" ) )

In [0]:
def sampling(dfo, freq = 'D',interpolate = False, method = 'linear'):
  df = dfo.copy()
  df.index = pd.to_datetime(df.index)
  df_interpol = df.resample(freq).mean()
  if interpolate:
    df_interpol[df.columns] = df_interpol[df.columns].interpolate(method=method)
  return df_interpol

In [0]:
# Aggregating data quarterly
data_agg = {}
for key,df in data_dict.items():
  data_agg[key]=sampling(df,'Q')

In [0]:
# first calcualte the percentage change
# relative change w.r.t. the market
df = data_agg['target'].copy()
df_relative_change = df.pct_change().div(df['market'].pct_change(), axis=0)

In [0]:
# Adding target variable to the data
data_agg['target_relative_change'] = df_relative_change.add_suffix('_rel')
data_agg['target_relative_change_plus_1'] = df_relative_change.shift(-1, axis = 0).add_suffix('_relplus1')
del data_agg['target']

In [0]:
# Creating dataframes dictionary with 
data_pc = {}
for key,df in data_agg.items():
  if key not in ['market','target_relative_change','target_relative_change_plus_1']:
    data_pc[key+'_pc'] = df.pct_change().add_suffix('_pc')

In [0]:
# Merging the two dictionaries
data_agg.update(data_pc)

In [10]:
combined = pd.concat(data_agg.values(), join='outer', axis=1)
combined.head(3)

,allEmployeesChemicals,capacityUtilization,corporateProfitsInventoryChem,industrialCapacityChem,industrialProdChem,industrialProductionChemProd,pPIChemicalManufac,pPIPaintMaterialsChem,quarterlyFinancialReportChem,ratioManufacturersInvtoShipChem,valueManufacturersShipmentsChem,cBOECrudeOilETFVolatily,cPIFuelOil,capUtilizationOilGasExtraction,crudeOilPricesWTI,employeesOilGasExtraction,globalBrentCrude,globalWTICrude,grossInputsRefineries,industrialProdCrudeOil,industrialProdDrilling,industrialProdMachinery,industrialProdOilGasExtraction,pPIDrillingOilGas,pPIOilGasExtraction,pPIOilGasMachinery,pPIPipelineTransCrudeOil,percentUtilizationRefineryOperableCapacity,wTISpotPriceFOB,airRevenueFreightMail,airTransportation,cPIAirlineFares,cPI_SA,domesticLoadFactor,enplaned,exportsTravel,internationalLoadFactor,pPAirlinePassengerServices,pPAirportParking,pPCarRental,...,pPIOilGasExtraction_pc,pPIOilGasMachinery_pc,pPIPipelineTransCrudeOil_pc,percentUtilizationRefineryOperableCapacity_pc,wTISpotPriceFOB_pc,airRevenueFreightMail_pc,airTransportation_pc,cPIAirlineFares_pc,cPI_SA_pc,domesticLoadFactor_pc,enplaned_pc,exportsTravel_pc,internationalLoadFactor_pc,pPAirlinePassengerServices_pc,pPAirportParking_pc,pPCarRental_pc,pPFlightBookings_pc,pPScheduledPassenger_pc,uS_Jet_Fuel_Spot_Price_pc,airlineGT_pc,consumerOpinionSurvey_pc,consumerPriceIndex_pc,currentAccount_SA_pc,gDP_QPCSA_pc,interbankRates90day_pc,leadingIndicatorsOECD_pc,m2_pc,pERMIT_pc,rDPI_pc,retailSales_pc,uMCSENT_pc,uS10YearBondYield_pc,unemploymentRate_pc,allEmployeesFinanceInsurance_pc,exportPriceIndPharma_pc,grossOutputFinanceInsurance_pc,industrialProdPharma_pc,lifeInsuranceCompaniesGeneralAccounts_pc,personalConsumptionExpendituresFinServInsurance_pc,producerPriceIndexPharma_pc
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2009-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-31,789.333333,69.2930,42.578,145.619900,100.904467,105.1843,230.666667,219.566667,187615.0,1.213333,56323.666667,NaN,278.053667,94.016033,NaN,156.533333,76.666388,78.645208,14320.666667,83.359733,69.523933,60.676900,83.987667,315.633333,259.933333,246.833333,246.333333,81.4,78.807049,2.910235e+06,462.700000,271.912,217.374000,81.666667,47455.666667,8461.333333,81.833333,101.666667,NaN,202.833333,...,NaN,NaN,NaN,NaN,-0.007343,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.014958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-06-30,786.833333,70.3383,61.238,144.716567,101.790433,104.3628,233.366667,219.666667,197891.0,1.216667,57949.666667,NaN,272.069000,93.550400,NaN,158.133333,78.672590,77.863506,15665.000000,82.737767,71.970067,63.577233,84.259133,318.433333,231.166667,247.066667,255.266667,89.2,77.816032,3.068738e+06,459.666667,275.668,217.297333,81.600000,54343.333333,8657.666667,81.800000,104.533333,NaN,181.433333,...,-0.110669,0.000945,0.036265,0.095823,-0.012575,0.054464,-0.006556,0.013813,-0.000353,-0.000816,0.145139,0.023204,-0.000407,0.028197,NaN,-0.105505,0.011628,0.029641,0.031207,0.026549,-0.000363,-0.802416,0.053995,0.647059,1.048387,0.002539,0.010498,-0.088191,0.016475,0.017036,0.000903,-0.062352,-0.020339,-0.002097,0.012123,0.003638,-0.01272,-0.027374,0.012691,0.002155


## Saving Data

In [0]:
import pickle

with open('combined.pickle', 'wb') as handle:
    pickle.dump(combined, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
from google.colab import files
files.download('combined.pickle')

## Correlations

In [0]:
def high_corr(dfo, subindustry,industries,quant_thresh = .90):
  df = dfo.copy()
  industries.remove(subindustry)
  df = df.loc[:,~df.columns.str.contains("^"+"|^".join(industries))] 
  #df = df.drop(columns=industries)
  corr = df.corr()
  df_corr = corr[[subindustry]]
  df_high_corr = df_corr[(df_corr[subindustry].abs() >= df_corr[subindustry].abs().quantile(quant_thresh))]
  return df_high_corr

In [14]:
for ind in data_agg['target_relative_change'].columns:
  if 'market' not in ind:
    industries = data_agg['target_relative_change'].columns.values.tolist().copy()
    df_high_corr = high_corr(combined,ind,industries)
    print(df_high_corr.sort_values(by=ind))
    print('-----')

                                               specialty_rel
consumerOpinionSurvey_pc                           -0.308360
uMCSENT_pc                                         -0.288240
capUtilizationOilGasExtraction_pc                  -0.282075
enplaned                                            0.260765
cPIAirlineFares_pc                                  0.266819
grossInputsRefineries                               0.269566
enplaned_pc                                         0.297446
pPCarRental_pc                                      0.320024
percentUtilizationRefineryOperableCapacity          0.377918
pPAirportParking_pc                                 0.407501
percentUtilizationRefineryOperableCapacity_pc       0.423444
grossInputsRefineries_pc                            0.445207
specialty_rel                                       1.000000
-----
                                                    non-specialty_rel
pERMIT_pc                                                   -0.444020


In [15]:
for ind in data_agg['target_relative_change_plus_1'].columns:
  if 'market' not in ind:
    industries = data_agg['target_relative_change_plus_1'].columns.values.tolist().copy()
    df_high_corr = high_corr(combined,ind,industries)
    print(df_high_corr.sort_values(by=ind))
    print('-----')

                                            specialty_relplus1
m2_pc                                                -0.557943
industrialProductionChemProd_pc                      -0.334007
unemploymentRate_pc                                  -0.330946
capacityUtilization_pc                               -0.311317
pPAirportParking_pc                                  -0.289631
industrialProdChem_pc                                -0.286598
ratioManufacturersInvtoShipChem_pc                   -0.260183
allEmployeesChemicals_pc                             -0.224288
percentUtilizationRefineryOperableCapacity           -0.222395
internationalLoadFactor                               0.270643
grossOutputFinanceInsurance_pc                        0.304561
consumerOpinionSurvey_pc                              0.502086
uMCSENT_pc                                            0.547792
specialty_relplus1                                    1.000000
-----
                                 non-specialty_re